# Example USage of LanceDB-haystack in a Haystack Pipeline

**Author:** Alan Meeson <am@carefullycalculated.co.uk>

**Date:** 2024-05-18

This notebook provides an example of using the LanceDB backed FTS and embedding DocumentStore with Haystack 2.x.

## Setup

In [1]:
import pyarrow as pa

from haystack import Document, Pipeline
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack_integrations.components.embedders.fastembed import FastembedDocumentEmbedder, FastembedTextEmbedder

from lancedb_haystack import LanceDBDocumentStore, LanceDBEmbeddingRetriever, LanceDBFTSRetriever

## Declare the document store and associated settings

In [2]:
lance_db_path = 'lancedb'
metadata_schema = pa.struct([
    ('author', pa.string()),
    ('title', pa.string()),    
    ('url', pa.string())
])
document_store = LanceDBDocumentStore(database=lance_db_path, table_name="documents", metadata_schema=metadata_schema, embedding_dims=384)


embedding_retriever = LanceDBEmbeddingRetriever(document_store)

In [3]:
embedder = FastembedDocumentEmbedder()
embedder.warm_up()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

## Do the Indexing

In [4]:
p = Pipeline()
p.add_component(instance=DocumentCleaner(), name="cleaner")
p.add_component(instance=DocumentSplitter(split_by="sentence", split_length=250, split_overlap=30), name="splitter")
p.add_component(instance=embedder, name="embedder")
p.add_component(instance=DocumentWriter(document_store=document_store), name="writer")
p.connect("cleaner.documents", "splitter.documents")
p.connect("splitter.documents", "embedder.documents")
p.connect("embedder.documents", "writer.documents")

🚅 Components
  - cleaner: DocumentCleaner
  - splitter: DocumentSplitter
  - embedder: FastembedDocumentEmbedder
  - writer: DocumentWriter
🛤️ Connections
  - cleaner.documents -> splitter.documents (List[Document])
  - splitter.documents -> embedder.documents (List[Document])
  - embedder.documents -> writer.documents (List[Document])

In [5]:
docs = [
    Document(
        content="A man must appear somewhat vain, who declares that he has been obliged to reject much useful information, for fear of increasing too much the size of his work: and yet manages to find room for a few pages of his own, by way of Preface: but lest the objects which the compiler of this little work has had in view should be mistaken, he finds it absolutely necessary to say a few words in explanation of them. This small collection of military memorandums was originally intended only for the compiler’s own pocket; to assist him in the execution of his duty: but it occurred to him, that many of his military friends stood in equal need of such an aid, and would willingly give a few shillings for what they would not be at the trouble of collecting. The compiler has seen young men, on their first entry into the regiment of artillery, give a guinea for manuscripts, which contained a very small part of the information offered in this little book.",
        meta={
            'title': 'The Bombardier, and Pocket Gunner.',
            'author':'Ralph Willet Adye',
            'url':'https://gutenberg.org/cache/epub/73283/pg73283-images.html'
        }
    ),
    Document(
        content="26th July, 1799.—Left H. H. this morning to make a tour into Wales and the west of England. Ld. H. went to dine at St. Anne’s, and Drew, Charles, and myself came to this place (General Fitzpatrick’s, Sunninghill) to meet him. On Saturday, ye 27th, we arrived to dinner at Oxford. Close by the General’s, upon Ascot Heath, there is a camp. The cavalry were watered whilst we went on the road, a pretty incident; their brilliant coats contrasted with the dusky brown of the heath, and produced a very lively effect. A cross road to Henley; just as we entered the town we passed under Park Place, a late purchase of Ld. Malmesbury’s. Somewhere near Henley is the highest ground south of Trent. On ye 28th went from Oxford, by Chapel House, to Warwick. I could not help laughing at the event of our going, because the subject of seeing the Castle without being obliged by urgent solicitations to make a long visit there (as Ld. H. was sure we should be very much pressed) had occupied his thoughts for many hours. Ld. H. had written letters to Ly. W. and Mrs. Smith, so calculated as to make a refusal of the pressing invitations (which he dreaded having) quite civil; but we need not have been embarrassed. So far from being solicited to stay, the whole family flew to their fortresses in some battlemented towers, gave out[2] that they were walking, and shunned us. It was a most fortunate escape to me, tho’ I believe Ld. H. was vexed at the impertinence of the proceeding. The Castle is magnificent, decorated with ornaments appropriate to the building, and in the very best taste possible. His taste in virtu is better than in breeding, as he manifested a gross deficiency of that quality to us.",
        meta={
            'title': 'The journal of Elizabeth Lady Holland',
            'author':'Elizabeth Lady Holland',
            'url':'https://gutenberg.org/cache/epub/73280/pg73280-images.html'
        }
    )    
]


In [6]:
result = p.run({"cleaner": {"documents": docs}})

Calculating embeddings: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  7.33it/s]


In [7]:
result

{'writer': {'documents_written': 2}}

## Query

### FTS

In [8]:
fts_retriever = LanceDBFTSRetriever(document_store)
query_pipeline = Pipeline()
query_pipeline.add_component("fts_retriever", fts_retriever)

In [9]:
table = document_store.db.open_table(document_store._table_name)
table.create_fts_index("content", replace=True)

In [10]:
query = "wales"
results = query_pipeline.run({"fts_retriever": {"query": query}})
print(f"Number of Results: {len(results['fts_retriever']['documents'])}")
results


Number of Results: 1


{'fts_retriever': {'documents': [Document(id=5de0a36f2936e54b099798a464ebfa165fa07f19ff3bb24be84649175fed1050, content: '26th July, 1799.—Left H. H. this morning to make a tour into Wales and the west of England. Ld. H. w...', meta: {'author': 'Elizabeth Lady Holland', 'title': 'The journal of Elizabeth Lady Holland', 'url': 'https://gutenberg.org/cache/epub/73280/pg73280-images.html'}, score: 0.6513087749481201, embedding: vector of size 384)]}}

In [11]:
query = "vain"
results = query_pipeline.run({"fts_retriever": {"query": query}})
print(f"Number of Results: {len(results['fts_retriever']['documents'])}")
results

Number of Results: 1


{'fts_retriever': {'documents': [Document(id=50bd7da4a4ccf12d4532be79fee2874d1d8846d1bbde305908ffb6718cf35aaa, content: 'A man must appear somewhat vain, who declares that he has been obliged to reject much useful informa...', meta: {'author': 'Ralph Willet Adye', 'title': 'The Bombardier, and Pocket Gunner.', 'url': 'https://gutenberg.org/cache/epub/73283/pg73283-images.html'}, score: 0.7922531962394714, embedding: vector of size 384)]}}

In [12]:
document_store.filter_documents()

[Document(id=50bd7da4a4ccf12d4532be79fee2874d1d8846d1bbde305908ffb6718cf35aaa, content: 'A man must appear somewhat vain, who declares that he has been obliged to reject much useful informa...', meta: {'author': 'Ralph Willet Adye', 'title': 'The Bombardier, and Pocket Gunner.', 'url': 'https://gutenberg.org/cache/epub/73283/pg73283-images.html'}, embedding: vector of size 384),
 Document(id=5de0a36f2936e54b099798a464ebfa165fa07f19ff3bb24be84649175fed1050, content: '26th July, 1799.—Left H. H. this morning to make a tour into Wales and the west of England. Ld. H. w...', meta: {'author': 'Elizabeth Lady Holland', 'title': 'The journal of Elizabeth Lady Holland', 'url': 'https://gutenberg.org/cache/epub/73280/pg73280-images.html'}, embedding: vector of size 384)]

### Embedding

In [13]:
text_embedder = FastembedTextEmbedder()
text_embedder.warm_up()

In [14]:
query_pipeline2 = Pipeline()
query_pipeline2.add_component("text_embedder", text_embedder)
query_pipeline2.add_component("embedding_retriever", embedding_retriever)
query_pipeline2.connect("text_embedder.embedding", "embedding_retriever")

🚅 Components
  - text_embedder: FastembedTextEmbedder
  - embedding_retriever: LanceDBEmbeddingRetriever
🛤️ Connections
  - text_embedder.embedding -> embedding_retriever.query_embedding (List[float])

In [15]:
query = "tour of the country"
results = query_pipeline2.run({"text_embedder": {"text": query}})
print(f"Number of Results: {len(results['embedding_retriever']['documents'])}")
results

Calculating embeddings: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 37.37it/s]

Number of Results: 2


{'embedding_retriever': {'documents': [Document(id=5de0a36f2936e54b099798a464ebfa165fa07f19ff3bb24be84649175fed1050, content: '26th July, 1799.—Left H. H. this morning to make a tour into Wales and the west of England. Ld. H. w...', meta: {'author': 'Elizabeth Lady Holland', 'title': 'The journal of Elizabeth Lady Holland', 'url': 'https://gutenberg.org/cache/epub/73280/pg73280-images.html'}, score: 0.7192904353141785, embedding: vector of size 384),
   Document(id=50bd7da4a4ccf12d4532be79fee2874d1d8846d1bbde305908ffb6718cf35aaa, content: 'A man must appear somewhat vain, who declares that he has been obliged to reject much useful informa...', meta: {'author': 'Ralph Willet Adye', 'title': 'The Bombardier, and Pocket Gunner.', 'url': 'https://gutenberg.org/cache/epub/73283/pg73283-images.html'}, score: 1.0694129467010498, embedding: vector of size 384)]}}

In [16]:
query = "reject useful information"
results = query_pipeline2.run({"text_embedder": {"text": query}, "embedding_retriever": {"top_k": 1}})
print(f"Number of Results: {len(results['embedding_retriever']['documents'])}")
results

Calculating embeddings: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 30.93it/s]

Number of Results: 1


{'embedding_retriever': {'documents': [Document(id=50bd7da4a4ccf12d4532be79fee2874d1d8846d1bbde305908ffb6718cf35aaa, content: 'A man must appear somewhat vain, who declares that he has been obliged to reject much useful informa...', meta: {'author': 'Ralph Willet Adye', 'title': 'The Bombardier, and Pocket Gunner.', 'url': 'https://gutenberg.org/cache/epub/73283/pg73283-images.html'}, score: 0.739778459072113, embedding: vector of size 384)]}}